In [1]:
!python -m pip install --user -q pypdf pdfplumber pandas regex


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from pathlib import Path

p1 = Path("/storage/brno2/home/filipja2/esg-gather/data/raw/Allianz_Group_Sustainability_Report_2023-web.pdf")
p2 = Path("/storage/brno2/home/filipja2/esg-gather/data/raw/en-allianz-group-annual-report-2024.pdf")
p3 = Path("/storage/brno2/home/filipja2/esg-gather/data/raw/en-allianz-group-non-financial-supplement-2024.pdf")  # pokud existuje

pdfs = [p for p in [p1, p2, p3] if p.exists()]
pdfs


[PosixPath('/storage/brno2/home/filipja2/esg-gather/data/raw/Allianz_Group_Sustainability_Report_2023-web.pdf'),
 PosixPath('/storage/brno2/home/filipja2/esg-gather/data/raw/en-allianz-group-annual-report-2024.pdf'),
 PosixPath('/storage/brno2/home/filipja2/esg-gather/data/raw/en-allianz-group-non-financial-supplement-2024.pdf')]

In [1]:
import re, pdfplumber, pandas as pd
from pathlib import Path

def extract_lines(pdf_path, x_tol=2, y_tol=2):
    lines = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            txt = page.extract_text(x_tolerance=x_tol, y_tolerance=y_tol) or ""
            for ln in txt.splitlines():
                ln = (ln.replace("\u00A0", " ")
                        .replace("CO₂e", "CO2e")
                        .replace("–", "-"))  # občas en-dash
                ln = re.sub(r"\s+", " ", ln).strip()
                if ln:
                    lines.append(ln)
    return lines

def to_float(num_str, unit="t"):
    s = num_str.strip().replace(" ", "")
    # heuristika EU/US oddělovačů
    if "," in s and "." in s:
        last = max(s.rfind(","), s.rfind("."))
        s = s.replace(".", "").replace(",", ".") if s[last] == "," else s.replace(",", "")
    elif "," in s:
        s = s.replace(",", "") if (s.count(",")==1 and len(s.split(",")[-1])==3) else s.replace(",", ".")
    # sanity: vyřaď „vlak“ čísel
    if sum(ch.isdigit() for ch in s) > 9:
        return None
    try:
        val = float(s)
    except:
        return None

    unit = (unit or "t").lower()
    # hmotnostní jednotky
    if unit.startswith("k") and "g" not in unit:    # "kt"
        val *= 1_000
    if "mn" in unit and "kg" not in unit:          # "mn t"
        val *= 1_000_000
    if "kg" in unit:                               # "kg"
        val /= 1_000
    return val

RE_CO2E = r"CO\s*2\s*e"

# --- Scope 1/2/3 ---
# Obecný scope (pokud nenajdeme spec. varianty)
PAT_SCOPE = re.compile(
    rf"\bScope\s*(?P<scope>[123])\b.*?"
    rf"(?P<val>\d{{1,3}}(?:[.,\s]\d{{3}})*(?:[.,]\d+)?|\d+(?:[.,]\d+)?)\s*"
    rf"(?P<unit>(?:mn\s*)?t|kt|t|kg)?\s*{RE_CO2E}",
    re.I,
)

# Scope 2 – preferuj market-based
PAT_SCOPE2_MARKET = re.compile(
    rf"\bScope\s*2\b.*?\bmarket[-\s]*based\b.*?"
    rf"(?P<val>\d{{1,3}}(?:[.,\s]\d{{3}})*(?:[.,]\d+)?|\d+(?:[.,]\d+)?)\s*"
    rf"(?P<unit>(?:mn\s*)?t|kt|t|kg)?\s*{RE_CO2E}",
    re.I,
)
PAT_SCOPE2_LOCATION = re.compile(
    rf"\bScope\s*2\b.*?\blocation[-\s]*based\b.*?"
    rf"(?P<val>\d{{1,3}}(?:[.,\s]\d{{3}})*(?:[.,]\d+)?|\d+(?:[.,]\d+)?)\s*"
    rf"(?P<unit>(?:mn\s*)?t|kt|t|kg)?\s*{RE_CO2E}",
    re.I,
)

# --- Total GHG ---
PAT_TOTAL = re.compile(
    rf"\b(?:Total|Celkem)\b.*?\b(?:GHG|greenhouse gas|emise)\b.*?"
    rf"(?P<val>\d{{1,3}}(?:[.,\s]\d{{3}})*(?:[.,]\d+)?|\d+(?:[.,]\d+)?)\s*"
    rf"(?P<unit>(?:mn\s*)?t|kt|t|kg)?\s*{RE_CO2E}",
    re.I,
)

# --- Intenzity proprietárních investic (víc variant) ---
# tCO2e / € mn  |  tCO2e per € mn  |  tCO2e/€ m
PAT_INV_INT_MN = re.compile(
    rf"(?:t\s*CO2e|CO2e\s*\(t\))\s*(?:/|per)\s*(?:€|EUR)\s*(?:mn|m)\b.*?"
    rf"(?P<val>\d+(?:[.,]\d+)?)",
    re.I,
)
# tCO2e / € bn
PAT_INV_INT_BN = re.compile(
    rf"(?:t\s*CO2e|CO2e\s*\(t\))\s*(?:/|per)\s*(?:€|EUR)\s*bn\b.*?"
    rf"(?P<val>\d+(?:[.,]\d+)?)",
    re.I,
)
# kgCO2e / € mn
PAT_INV_INT_KG_MN = re.compile(
    rf"(?:kg\s*CO2e)\s*(?:/|per)\s*(?:€|EUR)\s*(?:mn|m)\b.*?"
    rf"(?P<val>\d+(?:[.,]\d+)?)",
    re.I,
)

def scan_metrics_by_lines(lines):
    hits = {}
    n = len(lines)
    for i in range(n):
        window = lines[i] + (" " + lines[i+1] if i+1 < n else "")

        # Scope 2 location/market-based (uložíme zvlášť; market má prioritu i do scope2_co2e_t)
        for m in PAT_SCOPE2_LOCATION.finditer(window):
            v = to_float(m.group("val"), m.group("unit") or "t")
            if v is not None:
                hits["scope2_location_t"] = v
        for m in PAT_SCOPE2_MARKET.finditer(window):
            v = to_float(m.group("val"), m.group("unit") or "t")
            if v is not None:
                hits["scope2_market_t"] = v
                hits["scope2_co2e_t"] = v  # defaultní scope2 = market-based, pokud je k dispozici

        # Obecný Scope 1/2/3 (nenahrazuj scope2_co2e_t, pokud už máme market-based)
        for m in PAT_SCOPE.finditer(window):
            scope = m.group("scope")
            v = to_float(m.group("val"), m.group("unit") or "t")
            if v is None:
                continue
            key = f"scope{scope}_co2e_t"
            if key == "scope2_co2e_t" and "scope2_co2e_t" in hits:
                continue
            hits[key] = v

        # Total
        for m in PAT_TOTAL.finditer(window):
            v = to_float(m.group("val"), m.group("unit") or "t")
            if v and 0 < v < 1e9:
                hits.setdefault("total_co2e_t", v)

        # Intenzity
        m = PAT_INV_INT_MN.search(window)
        if m:
            v = to_float(m.group("val"), "t")  # už je to „t / € mn“
            if v:
                hits["investments_intensity_t_per_eur_mn"] = v
        m = PAT_INV_INT_BN.search(window)
        if m:
            v = to_float(m.group("val"), "t")
            if v:
                hits["investments_intensity_t_per_eur_mn"] = v / 1000.0  # převod z „/ € bn“ na „/ € mn“
        m = PAT_INV_INT_KG_MN.search(window)
        if m:
            v = to_float(m.group("val"), "kg")  # kg -> t
            if v:
                hits["investments_intensity_t_per_eur_mn"] = v

    # sanity: pokud máme market/location, ale chybí obecný scope2, nastav ho
    if "scope2_co2e_t" not in hits:
        if "scope2_market_t" in hits:
            hits["scope2_co2e_t"] = hits["scope2_market_t"]
        elif "scope2_location_t" in hits:
            hits["scope2_co2e_t"] = hits["scope2_location_t"]

    # sanity: total aspoň >= (scope1+scope2), jinak ho raději zahoď
    s12 = sum(hits.get(k, 0) for k in ["scope1_co2e_t","scope2_co2e_t"])
    tot = hits.get("total_co2e_t")
    if tot and tot < s12:
        hits.pop("total_co2e_t", None)

    return hits

def debug_find(lines, pattern, limit=8):
    out = []
    for i, ln in enumerate(lines):
        if re.search(pattern, ln, re.I):
            ctx = " | ".join(lines[max(0,i-1):min(len(lines), i+2)])
            out.append(ctx)
            if len(out) >= limit: break
    return out


In [8]:
import re
from typing import Dict, List

NUM = r"(-?\d{1,3}(?:[.,]\d{3})*(?:[.,]\d+)?|-?\d+(?:[.,]\d+)?)"

def first_number_after(text: str, label_pat: str) -> str | None:
    """
    V 'text' najde první číslo za 'label_pat' (regexem).
    Vrací string s číslem, nebo None.
    """
    m = re.search(label_pat + r"\s+" + NUM, text, flags=re.I)
    return m.group(1) if m else None

def current_section_tracker(lines: List[str]) -> List[str]:
    """Vrátí pro každý řádek odhad sekce: 'ownops', 'proprietary', nebo ''."""
    section = ""
    tags = []
    for ln in lines:
        low = ln.lower()
        if "proprietary investments ghg emissions" in low:
            section = "proprietary"
        elif "own operations" in low:
            section = "ownops"
        # drobný reset při nové tabulce/záhlaví
        if "tables of the sustainability statement" in low:
            section = ""
        tags.append(section)
    return tags

# labely, které chceme chytat (omezíme se na own operations, ať se nám to „nemísí“)
ROW_PATTERNS = {
    "ownops_scope1_t": r"Gross\s+Scope\s*1\s+GHG\s+emissions(?!.*per\s+employee)",
    "ownops_scope2_location_t": r"Gross\s+location[-\s]*based\s+Scope\s*2\s+GHG\s+emissions(?!.*per\s+employee)",
    "ownops_scope2_market_t": r"Gross\s+market[-\s]*based\s+Scope\s*2\s+GHG\s+emissions(?!.*per\s+employee)",
    "ownops_scope3_t": r"Scope\s*3\s+GHG\s+emissions(?!.*per\s+employee)",
    "ownops_total_location_t": r"Total\s+own\s+operations\s+GHG\s+emissions\s*\(location[-\s]*based\)(?!.*per\s+employee)",
    "ownops_total_market_t": r"Total\s+own\s+operations\s+GHG\s+emissions\s*\(market[-\s]*based\)(?!.*per\s+employee)",
}

# volitelně: totéž pro proprietary investments (když budeš chtít)
PROP_PATTERNS = {
    "propinv_total_t": r"Proprietary\s+investments\s+GHG\s+emissions\s*\(Scope\s*1-3\)\s*-\s*TOTAL",
}

def parse_supplement_ownops(lines: List[str]) -> Dict[str, float]:
    """
    Z Non-Financial Supplementu vytáhne own-operations Scope1/2/3 a Total (loc/market).
    Bere první číslo po labelu jako 'aktuální rok' (většinou 2024).
    """
    hits: Dict[str, float] = {}
    tags = current_section_tracker(lines)

    for i, ln in enumerate(lines):
        sec = tags[i]
        row = ln

        # Předzpracování: spoj okno s následujícím řádkem (často jsou čísla až v dalším řádku)
        if i + 1 < len(lines):
            row_next = row + " " + lines[i+1]
        else:
            row_next = row

        if sec == "ownops":
            for key, pat in ROW_PATTERNS.items():
                m = re.search(pat, row, flags=re.I)
                if not m:
                    # někdy je label na konci řádku a čísla až na dalším
                    m = re.search(pat, row_next, flags=re.I)
                    if not m:
                        continue
                    txt = row_next
                else:
                    txt = row

                # první číslo po labelu = aktuální rok (vlevo v tabulce)
                # používáme stejný regex jako NUM, ale jen první výskyt ZA labelem
                # -> pomůže nám pomocná funkce:
                val_str = first_number_after(txt, pat)
                if val_str is None:
                    # fallback: hledej číslo v 'txt' ručně
                    mnum = re.search(pat + r"\s+" + NUM, txt, flags=re.I)
                    val_str = mnum.group(1) if mnum else None

                if val_str:
                    val = to_float(val_str, "t")  # jednotky bývají "t in CO2e" v záhlaví
                    if val is not None:
                        hits[key] = val

        elif sec == "proprietary":
            for key, pat in PROP_PATTERNS.items():
                if re.search(pat, row, flags=re.I) or re.search(pat, row_next, flags=re.I):
                    # po tomto labelu většinou rovnou následují čísla
                    mnum = re.search(pat + r".*?" + NUM, row_next, flags=re.I)
                    if not mnum:
                        mnum = re.search(pat + r".*?" + NUM, row, flags=re.I)
                    if mnum:
                        val = to_float(mnum.group(1), "t")
                        if val is not None:
                            hits[key] = val

    # sanity: když máme market i location, a chybí 'scope2_co2e_t', doplň market jako default
    if "ownops_scope2_market_t" in hits and "ownops_scope2_t" not in hits:
        hits["ownops_scope2_t"] = hits["ownops_scope2_market_t"]

    return hits


In [9]:
# cesty – použij ten svůj Supplement 2024
p3 = Path("/storage/brno2/home/filipja2/esg-gather/data/raw/en-allianz-group-non-financial-supplement-2024.pdf")
lines_p3 = extract_lines(p3)

hits_p3 = parse_supplement_ownops(lines_p3)
hits_p3

{'propinv_total_t': 202.0,
 'ownops_scope1_t': 36160.9,
 'ownops_scope2_location_t': 118928.5,
 'ownops_scope3_t': 102629.3,
 'ownops_scope2_market_t': 5947.0,
 'ownops_total_location_t': 257718.7,
 'ownops_total_market_t': 144737.2,
 'ownops_scope2_t': 5947.0}

In [10]:
records = []

# Sustainability 2023 a Annual 2024 nech zpracovat původním line-based skenerem, co už máš
for pdf in [p1, p2]:
    lines = extract_lines(pdf)
    base = scan_metrics_by_lines(lines)  # tvoje starší funkce (investments intensity apod.)
    base["source_pdf"] = pdf.name
    records.append(base)

# Supplement 2024: použij nový ownops parser
if p3.exists():
    lines = extract_lines(p3)
    extra = parse_supplement_ownops(lines)
    extra["source_pdf"] = p3.name
    records.append(extra)

df = pd.DataFrame(records)
out_csv = Path("/storage/brno2/home/filipja2/esg-gather/data/processed/esg_metrics.csv")
out_csv.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_csv, index=False)
df


,source_pdf,investments_intensity_t_per_eur_mn,propinv_total_t,ownops_scope1_t,ownops_scope2_location_t,ownops_scope3_t,ownops_scope2_market_t,ownops_total_location_t,ownops_total_market_t,ownops_scope2_t
0,Allianz_Group_Sustainability_Report_2023-web.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,en-allianz-group-annual-report-2024.pdf,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,en-allianz-group-non-financial-supplement-2024...,NaN,202.0,36160.9,118928.5,102629.3,5947.0,257718.7,144737.2,5947.0
